In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
if (path := 'C:/dev/projects/allocation') not in sys.path:
    sys.path.append(path)


import ipywidgets as w
import numpy as np
import pandas as pd
import seaborn as sns
from dask.distributed import Client
from pandas.tseries.offsets import DateOffset
from scipy.cluster.hierarchy import dendrogram
from scipy import stats
from matplotlib import pyplot as plt
from IPython.display import display
from dask.distributed import Client
pd.options.plotting.backend = "plotly"


from allocation import *
from allocation.strategies import TestStrategy
from allocation.portfolio import Portfolio
from allocation.risk_managers import ActiveSignalsRiskScaler, VaRLimitScaler
from allocation.estimators import *
from allocation.estimators.misc import EWM
from allocation.backtest import BTInspector

### 1. Data Loading
Only closing price and signals time series are needed

In [8]:
data_path = Path(r'M:\BHAI\Shared\MacroBot\Scrap_Results' if sys.platform == 'win32' else '/mnt/macrobot/Scrap_Results')

assets = pd.read_csv(data_path / 'assets.csv', index_col=0, parse_dates=True, dayfirst=True) \
    .drop(['AUD_Bond_10y_Fut', 'AUD_Equity_Fut'], axis=1) \
    .rename({'AUD_Bond_10y_Fut_shift1': 'AUD_Bond_10y_Fut', 'AUD_Equity_Fut_shift1': 'AUD_Equity_Fut'}, axis=1)
signals = pd.read_csv(data_path / 'Gio_Signals.csv', index_col=0, parse_dates=True, dayfirst=True) \
    .rename({'AUD_Bond_10y_Fut_shift1': 'AUD_Bond_10y_Fut'}, axis=1)
# assets = all_ohlc.loc[:, (signals.columns, 'close')].droplevel(1, axis=1) #.drop(['FX_NOK_TR', 'FX_MXN_TR'], axis=1)

## 2. Create the portfolio object

In [9]:
ptf = Portfolio.from_assets_and_signals(assets, signals)

## 3.Define all the allocation strategies to test

In [10]:
ewm = EWM(SampleStandardDeviation(), alpha=0.05)

als = dict(
    oon=OneOverN(),
    ir_std=InverseRisk(),
    ir_ewm=InverseRisk(measure=ewm),
    ir_cvar=InverseRisk(measure=SampleCVaR()),
    ir_var=InverseRisk(measure=SampleVaR()),
    # risk_budget=RiskBudgeting(),
    # max_sharpe=MaxRiskAdjusted(),
    # hrp=HRP(),
    # herc=HERC(),
)

In [11]:
client = Client()
client

<Client: 'tcp://127.0.0.1:59160' processes=4 threads=4, memory=25.77 GB>

In [12]:
bts = {}
for key, allocator in als.items():
    b = BackTest(
        ptf, 
        allocator,
        n_jobs=-1
        #risk_managers = [VaRLimitScaler(), ActiveSignalsRiskScaler()]
    )
    b.run(partial=True).adjust()
    bts[key] = b

In [ ]:
for key, bt in self.bts.items():
    bt.risk_managers = [VaRLimitScaler(), ActiveSignalsRiskScaler()]
    bt.reset_weights()
    

## Use the BTInspector to analyse the results

In [60]:
self = BTInspector(bts)
self

In [39]:
r = pd.concat({k: v.get_returns() for k, v in self.bts.items()}, axis=1)[:-20].loc['2012':]

In [56]:
        mean = r.mean()
        std = r.std()
        var = r.quantile(0.05)
        dd_dev = r[r < 0].std()
        df =         pd.DataFrame(dict(
            ann_ret=r.sum() / len(r) * 252,
            std=std,
            var5=var,
            cvar5=r[r <= var].mean(),
            sharpe_ratio=mean / std * (252 ** 0.5),
            sortino_ratio=mean / dd_dev,
            max_dd=drawdowns(r).min(),
            
        )).T

In [59]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: palegreen' if v else '' for v in is_max]

def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_min = s == s.min()
    return ['background-color: red' if v else '' for v in is_min]

df.style.apply(highlight_max).apply(highlight_min)

In [51]:
r.sum() / len(r) * 252

oon        0.129440
ir_std     0.111062
ir_ewm     0.110763
ir_cvar    0.111099
ir_var     0.110546
dtype: float64

In [40]:
mean = r.mean()
std = r.std()
var = r.quantile(0.05)
cvar = r[r <= var].mean()
sharpe_ratio = mean / std * (252 ** 0.5)
dd_dev = r[r<0].std()
sortino_ratio = mean / dd_dev

In [43]:
def drawdowns(returns, compound=False):
    levels = 1+ returns.cumsum() if compound else (1+returns).cumprod()
    return (levels / levels.cummax() - 1) if compound else levels - levels.cummax()

In [45]:
drawdowns().min()

oon       -0.084130
ir_std    -0.065268
ir_ewm    -0.064721
ir_cvar   -0.064865
ir_var    -0.064738
dtype: float64

In [ ]:
asset_subset = assets[['US_Equity_Fut', 'US_Bond_30y_Fut']]

In [ ]:
ptf2 = Portfolio.from_assets(asset_subset)

In [ ]:
ewm = EWM(SampleStandardDeviation(), alpha=0.05)

als = dict(
    oon=OneOverN(),
    ir_std=InverseRisk(),
    ir_ewm=InverseRisk(measure=ewm),
    ir_cvar=InverseRisk(measure=SampleCVaR()),
    ir_var=InverseRisk(measure=SampleVaR()),
    #risk_budget=RiskBudgeting(),
    #max_sharpe=MaxRiskAdjusted(),
    # hrp=HRP(),
    # herc=HERC(),
)

In [ ]:
bts = {}
for key, allocator in als.items():
    b = BackTest(
        ptf2, 
        allocator,
        first_training_size=500,
        n_jobs=-1
        #risk_managers = [VaRLimitScaler(), ActiveSignalsRiskScaler()]
    )
    b.run(partial=False).adjust()
    bts[key] = b

In [ ]:
self = BTInspector(bts)
self

In [ ]:
self = BTInspector(bts)
self

In [ ]:
std
var
cvar 
sharpe_ratio
ann_pnl
sortino_ratio
max_drawdown

In [ ]:
asset_subset.pct_change().rolling(250).std()['2017-07':][:20]

In [ ]:
p1 = self.bts['oon'].get_pnl().pct_change()

In [ ]:
p1.mean()/ p1.std() * (252**0.5)

In [ ]:
(asset_subset.pct_change() * self.bt.weights_).sum(axis=1).cumsum()

In [ ]:
self.bt.get_pnl(False)

In [ ]:
b = BackTest(
    ptf, 
    MaxRiskAdjusted(),
    risk_managers = [VaRLimitScaler(), ActiveSignalsRiskScaler()]
)

In [ ]:
asset_subset.pct_change()

In [ ]:
var = asset_subset.pct_change().dropna().rolling(500).quantile(0.05)

In [ ]:
(1 / var).divide((1 / var).sum(axis=1), axis=0).dropna()[48:]

In [ ]:
self.bts['ir_var'].weights_['2006-02-08':]

In [ ]:
assets['2004-12-10':]

In [ ]:
assets.pct_change().corr() > 0.3

In [ ]:
from scipy.cluster.hierarchy import linkage, leaves_list, fcluster, to_tree
from scipy.spatial.distance import pdist, squareform

In [ ]:
cdist = pdist(assets.pct_change().dropna().T, metric='correlation')

In [ ]:
assets.reset_index().melt(id_vars='index')

In [ ]:

(0.5 * (1 - assets.pct_change().corr()) ** 0.5)

In [ ]:
np.exp(-np.log(2) / 12)

In [ ]:
- np.log(2) / np.log(0.7)

In [ ]:
((assets.shape[1]) ** 2  - assets.shape[1]) / 2

In [ ]:
 assets.shape[1]

In [ ]:
from sklearn.preprocessing import PowerTransformer

In [ ]:
p = PowerTransformer()

In [ ]:
a = assets['US_Equity_Fut'].pct_change().dropna().to_frame()

In [ ]:
p.fit(a)

In [ ]:
p.lambdas_


In [ ]:
- np.log(2) / np.log(0.97)